In [29]:
import tkinter as tk
from tkinter import messagebox, scrolledtext
from tkinter import ttk
from openpyxl import load_workbook, Workbook
import datetime
import re  # Para separar valores en las celdas

# Variables globales
resultados_por_rol = {}
aplicativos = []
perfiles = []
nombre_archivo = "roles_aplicaciones.xlsx"

# Crear la ventana principal
root = tk.Tk()
root.title("Búsqueda en Catálogo para Códigos de Cargo")
root.geometry("1000x500")
root.resizable(True, True)

# Crear el Combobox para aplicativos
tk.Label(root, text="Aplicativo:").pack()
combo_aplicativos = ttk.Combobox(root)
combo_aplicativos.pack(fill="x", padx=10, pady=5)

# Campo para seleccionar perfil
tk.Label(root, text="Perfil a buscar:").pack()
entry_palabra = ttk.Combobox(root)
entry_palabra.pack(fill="x", padx=10, pady=5)

def guardar_en_excel():
    """Función para guardar los resultados en un archivo Excel."""
    palabra_buscada = entry_palabra.get().strip()
    aplicativo_buscado = combo_aplicativos.get()
    if not resultados_por_rol:
        messagebox.showerror("Error", "No hay resultados para generar el reporte.")
        return

    wb = Workbook()
    ws = wb.active
    ws.title = "Reporte"

    # Crear encabezados
    encabezados = [f"Rol que contiene '{palabra_buscada}' en '{aplicativo_buscado}'"]
    # Agregar encabezados para los códigos de cargo
    max_codigos = max(len(codigos) for codigos in resultados_por_rol.values())
    for i in range(max_codigos):
        encabezados.append(f"Código Cargo {i + 1}")
    
    ws.append(encabezados)

    # Escribir los datos en el archivo Excel
    for rol_value, codigos in resultados_por_rol.items():
        fila = [rol_value]  # Comienza con el rol
        fila.extend(codigos)  # Añade todos los códigos de cargo
        ws.append(fila)

    # Guardar el archivo con un nombre único
    fecha_actual = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    nombre_archivo = f"CodigoRol_{fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    messagebox.showinfo("Éxito", f"El reporte ha sido guardado como '{nombre_archivo}'.")

def cargar_aplicativos_y_perfiles():
    try:
        workbook = load_workbook(filename=nombre_archivo)
        sheet = workbook.worksheets[0]  # Acceder a la primera hoja
        max_row = sheet.max_row

        global aplicativos, perfiles
        aplicativos = set()
        perfiles = set()

        for i in range(2, max_row + 1):
            # Cargar aplicativos
            if sheet[f"E{i}"].value:
                palabras_aplicativo = [palabra.strip() for palabra in re.split(r'[;, ]+', sheet[f"E{i}"].value)]
                aplicativos.update(palabras_aplicativo)

            # Cargar perfiles
            if sheet[f"G{i}"].value:
                palabras_perfil = [palabra.strip() for palabra in re.split(r'[;, ]+', sheet[f"G{i}"].value)]
                perfiles.update(palabras_perfil)

        # Actualizar los combobox en la GUI
        combo_aplicativos['values'] = sorted(aplicativos)
        entry_palabra['values'] = sorted(perfiles)

    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error al cargar datos: {e}")

def buscar_datos():
    global resultados_por_rol
    palabra_buscada = entry_palabra.get().strip()
    aplicativo_buscado = combo_aplicativos.get()

    if not palabra_buscada or not aplicativo_buscado:
        messagebox.showerror("Error", "Debe seleccionar un aplicativo y un perfil.")
        return

    try:
        # Cargar el archivo Excel
        workbook = load_workbook(filename=nombre_archivo)
        sheet = workbook.worksheets[0]  # Acceder a la primera hoja

        # Columnas relevantes
        col_rol = "B"
        col_aplicacion = "E"
        col_perfil = "G"
        col_codigo_cargo = "K"

        max_row = sheet.max_row

        # Filtrar filas donde el aplicativo y perfil coincidan exactamente
        filas_relevantes = []
        for i in range(2, max_row + 1):
            aplicativo_actual = str(sheet[f"{col_aplicacion}{i}"].value or "").strip()
            perfil_actual = str(sheet[f"{col_perfil}{i}"].value or "").strip()
            perfiles_list = re.split(r'[;, ]+', perfil_actual)  # Separar por comas, punto y coma y espacios

            if aplicativo_buscado in re.split(r'[;, ]+', aplicativo_actual):
                # Verificar si la palabra buscada es un perfil exacto
                if palabra_buscada in perfiles_list:
                    rol = sheet[f"{col_rol}{i}"].value
                    if rol:  # Solo si hay un rol
                        filas_relevantes.append(rol)

        # Obtener todos los códigos de cargo para los roles encontrados
        resultados_por_rol.clear()
        for rol in set(filas_relevantes):  # Usar set para evitar duplicados
            for i in range(2, max_row + 1):
                aplicativo_actual = str(sheet[f"{col_aplicacion}{i}"].value or "").strip()
                rol_actual = sheet[f"{col_rol}{i}"].value

                # Si el rol actual coincide
                if rol_actual == rol:
                    codigo_cargo = sheet[f"{col_codigo_cargo}{i}"].value
                    
                    # Convertir a entero y eliminar comas
                    try:
                        codigo_cargo = int(str(codigo_cargo).replace(",", ""))
                    except ValueError:
                        continue  # Ignorar si no se puede convertir
                    
                    if rol not in resultados_por_rol:
                        resultados_por_rol[rol] = []
                    resultados_por_rol[rol].append(codigo_cargo)

        # Mostrar resultados en la ventana
        resultado_text.config(state=tk.NORMAL)
        resultado_text.delete("1.0", tk.END)

        if resultados_por_rol:
            resultado_text.insert(tk.END, f"Aplicativo: {aplicativo_buscado}\nPerfil: {palabra_buscada}\n\n")
            for rol, codigos in resultados_por_rol.items():
                resultado_text.insert(tk.END, f"Rol: {rol}\n" + "\n".join(f"Código Cargo: {codigo}" for codigo in codigos) + "\n\n")
            btn_generar_reporte.config(state=tk.NORMAL)
        else:
            resultado_text.insert(tk.END, "No se encontraron resultados.")
            btn_generar_reporte.config(state=tk.DISABLED)

        resultado_text.config(state=tk.DISABLED)

    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error al buscar datos: {e}")

# Cargar aplicativos y perfiles al iniciar
cargar_aplicativos_y_perfiles()

# Área de texto para mostrar resultados
frame_resultado = tk.Frame(root)
frame_resultado.pack(fill="both", expand=True, padx=10, pady=5)

resultado_text = scrolledtext.ScrolledText(frame_resultado, wrap="word", height=10)
resultado_text.pack(fill="both", expand=True)
resultado_text.config(state=tk.DISABLED)

# Botón para buscar
tk.Button(root, text="Buscar", command=buscar_datos).pack(pady=5)

# Botón para generar el reporte
btn_generar_reporte = tk.Button(root, text="Generar Reporte", command=guardar_en_excel, state=tk.DISABLED)
btn_generar_reporte.pack(pady=5)

# Iniciar la ventana principal
root.mainloop()